#  <center> Uvod </center> 

Stabla odluke se u mašinskom učenju upotrebljavaju u svrhu klasifikacije željenih instanci, odnosno određivanja vrednosti nepoznatog klasnog atributa.

Prednost korišćenja stabla odluke za klasifikaciju je jednostavnija realizacija, ali i razumevanje. Naime, pri vizuelizaciji stabla odluke, prateći svaku putanju od korenog čvora pa do svakog lista, vrlo jednostavno, govornim jezikom može se pročitati kako će se vršiti klasifikacija i koji je redosled pitanja na koji će algoritam odgovarati pri klasifikaciji instanci.

Nedostaci korišćenja stabla odluke ogledaju se u tome da se generisano stablo može previše "navići" na trening skup koji se koristio pri generisanju stabla odluke. Ova pojava se naziva prenaučenje, odnosno *overfitting*. Posledica prenaučenja je ta da će stablo davati odlične rezultate pri klasifikaciji instanci koje čine trening skup, a davati loše rezultate pri klasifikaciji instanci koje ne pripadaju trening skupu i nisu učestvovale pri generisanju stabla odluke (npr. instance koje se nalaze u test skupu). Do prenaučenja dolazi kada se svaki list stabla odluke generiše tek kada se utvrdi homogenost skupa. Da bi se izbeglo prenaučenje stabla odluke, koristi se odsecanje stabla, odnosno *pruning*. Pod odsecanjem, podrazumevaju se sve taktike koje će smanjiti veličinu, odnosno kompleksnost stabla odluke i smanjiti efekat prenaučenja. Jedna od taktika koje se mogu primenjivati prilikom odsecanja stabla, može biti ograničenje skupa instanci ispod koje se dalje ne vrši razbijanje skupa, čak i u slučaju da skup nije homogen, tako da se list generiše i u situaciji kada entropija datog skupa nije jednaka nuli (klasifikacija će biti izvršena na osnovu klase kojoj pripada najveći broj instanci iz skupa). Druga taktika može biti ograničenje dubine stabla odluke itd.

*Random forest* algoritam, pri klasifikaciji koristi grupu, odnosno ansambl stabala odluke. Svako stablo koje čini ansambl vrši svoju individualnu klasifikaciju, a do konačne klasifikacije dolazi se "brojanjem glasova". Ona klasa koja je "izglasana" najveći broj puta, računa se kao konačna klasa date instance čija je klasifikacija vršena.
U *random forest* algoritmu svako stablo se generiše u potpunosti, odnosno ne koristi se odsecanje individualnog stabla. Efekat prenaučenja kome je podložno individualno stablo odluke smanjuje se korišćenjem grupe različitih stabala. Svako stablo se generiše na osnovu instanci koje se biraju nasumično iz inicijalnog skupa podataka, a za razliku od korišćenja individualnog stabla odluke, gde instance ne smeju biti ponavljanje u trening skupu, kod *random forest* algoritma to je dozvoljeno (takav skup se naziva **bootsrapped** skup). 

Pod odsecanjem u *random forest* terminologiji podrazumeva se smanjenje broja stabala koja čine ansambl. Naime, na prvi pogled može se učiniti da se povećanjem broja stabala u ansamblu vrši i preciznija klasifikacija instanci. To će i biti slučaj, ali samo do određene veličine ansambla. Nakon ovog broja, odnosno generisanjem većeg broja stabala odluke, tačnost klasifikacije će se smanjivati.

Nije poznat tačan kriterijum kako doći do optimalnog broja stabala u ansamblu za koji će klasifikacija biti najtačnija, već programeri vrše testiranje menjanjem parametara (povećanje i smanjenje broja generisanih stabala).
U daljem izveštaju sledi implementacija *random forest* algoritma, kao i objašnjenje svakog dela koda. Algoritam vrši klasifikaciju instanci (osoba) na one koje imaju, odnosno one koje nemaju šećernu bolest. Skup podataka preuzet je sa adrese: https://www.mldata.io/dataset-details/pima_native_american_diabetes/, a podaci su prikupljeni uzimanjem uzoraka krvi američkih starosedelaca.

Najpre je potrebno učitati pomoćne biblioteke. 

In [2014]:
import numpy as np
import pandas as pd
import random
from pprint import pprint
from sklearn.feature_selection import SelectKBest, chi2

Zatim, učitava se *csv* fajl koji sadrži podatke. Filtriranje kolona, koje se postiže navođenjem imena istih, upotrebjeno je u svrhu korektnog sortiranja kolona, za potrebe pomoćnih funkcija, od kojih pojedine (kao na primer funkcija za računanje entropije) zahteva da se atribut, odnosno kolona na osnovu koje se klasifikacija vrši, nađe kao poslednja kolona u *data frame*-u.   

Sa druge strane, ovo su kolone koje su relevantne za kreiranje modela, odnosno koje će učestvovati prilikom klasifikacije željene instance (samim tim obezbeđeno je filtriranje kolona koje ne nose nikakve relevantne informacije za kreiranje modela kao što može biti kolona *ID* na primer). 

In [2015]:
df = pd.read_csv('pima_native_american_diabetes_weka_dataset.csv')
df = df[['times_pregnant', 'plasma_glucose', 'diastolic_blood_pressure',
'tricep_skin_fold_thickness', 'serum_insulin', 'body_mass_index',
 'diabetes_pedigree_function', 'age', 'class']]

*Random forest* algoritam treba da ima implementiran mehanizam koji može da radi sa nepotpunim podacima. U tu svrhu, pre generisanja stabala, potrebno je pretprocesirati inicijalne podatke. U konkretnom primeru, odnosno skupu, nepotpuni podaci su označeni nulom, tako da funkcija **replace_missing_values** vrši zamenu nula u svim kolonama prosečnim vrednostima koje se javljaju u odgovarajućoj koloni (izuzev u kolonama *times_pregnant* i *class*). 

In [2016]:
def replace_missing_values(df):

    averige = df.mean()
    df = df.replace({'plasma_glucose': {0: averige['plasma_glucose']},
                     'diastolic_blood_pressure': {0: averige['diastolic_blood_pressure']},
                     'tricep_skin_fold_thickness': {0: averige['tricep_skin_fold_thickness']},
                     'serum_insulin': {0: averige['serum_insulin']},
                     'body_mass_index': {0: averige['body_mass_index']},
                     'diabetes_pedigree_function': {0: averige['diabetes_pedigree_function']},
                     'age': {0: averige['age']}})

    df = df.replace({'class': {0: 'negative'}})
    df = df.replace({'class': {1: 'positive'}})
    return df

Funkcija **determine_best_features** vrši sortiranje atributa od onog koji je najbitniji pa prema manje bitnim atributima, gledajući prema atributu koji je potrebno odrediti (klasnom atributu). Funkcija koristi pomoć **sklearn** biblioteke, koja ima razne načine na koje može da utvrdi relevantnost atributa na osnovu raznih statističkih testova. Konkretno, u funkciji je upotrebjen **chi squared** test.

Cilj testiranja relevantnosti atributa je eliminisanje onih nebitnih, odnosno manje bitnih kolona u cilju smanjenja robusnosti podataka i poboljšanja performansi aplikacije.

Funkcija vraca dvodimenzionalno **numpy** polje, gde je prvi element svake vrste naziv atributa, a drugi skor koji je atribut postigao na testu.

In [2017]:
def determine_best_features(df):
    # Split the data into input and target
    X = df.iloc[:, 0:8]
    Y = df.iloc[:, 8]
    # We will select the features using chi square
    test = SelectKBest(score_func=chi2, k=8)
    # Fit the function for ranking the features by score
    fit = test.fit(X, Y)
    best_scores = fit.scores_
    columns = X.columns.values
    #print(columns)
    #print(best_scores)
    columns_dataframe = pd.DataFrame(columns)
    best_scores_dataframe = pd.DataFrame(best_scores)
    joined_dataframe = pd.concat([columns_dataframe, best_scores_dataframe], axis=1)
    joined_dataframe.columns = ['features', 'scores']
    #print('udruzeno:')
    #print(joined_dataframe)
    sorted_joined_dataframe = joined_dataframe.nlargest(8, 'scores')
    return sorted_joined_dataframe.values

Funkcija **keep_most_relevant_features** vrši filtriranje prosleđenog skupa podataka, zadržavajući samo onoliko atributa (kolona) predstavljeno brojem koji se prosledi preko **number_of_features_to_keep** atributa. Atribut **best_features_sorted** sadrži sortirane atribute, počevši od najbitnijeg pa prema onim manje bitnim atributima.

Funkcija vraća filtriran skup podataka, odbacujući manje bitne kolone.

In [2018]:
def keep_most_relevant_features(df, number_of_features_to_keep, best_features_sorted):
    if number_of_features_to_keep > 0 and number_of_features_to_keep <= 8:
        list_of_features_to_keep = []
        for i in range(number_of_features_to_keep):
            list_of_features_to_keep.append(best_features_sorted[i, 0])
        list_of_features_to_keep.append('class')
        df = df[list_of_features_to_keep]
    #print(list_of_features_to_keep)
    #print(df.head())
    return df

Funkcija *train_test_split* vrši podelu inicijalnog skupa instanci na trening i test skup. Argumenti:
 - **df** - Inicijalni skup instanci,
 - **test_size** - Veličina test skupa. Ovu veličinu je potrebno zadavati u procentima.
 
 Funkcija vraća dve vrednosti: trening i test skup.

In [2019]:
def train_test_split(df, test_size):

    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)

    return train_df, test_df

random.seed(0)

Funkcija **check_purity**, vrši ispitivanje homogenosti skupa prosleđenog kao ulazni (**data**) argument. U slučaju da je prosleđeni skup homogen (vrednost entropije skupa jednaka 0), funkcija vraća *True*, a u slučaju da prosleđeni skup nije homogen (0 < entropija <=1 ), funkcija vraća *False*.

In [2020]:
def check_purity(data):

    label_column = data[:, -1]
    unique_classes = np.unique(label_column)
    
    if len(unique_classes) == 1:
        return True
    else:
        return False


Funkcija **classify_data**, vrši klasifikaciju prosleđenog skupa (**data** argument) na osnovu poslednje kolone, odnosno atributa. Funkcija se ne bavi ispitivanjem homogenosti prosleđenog skupa, već vrši klasifikaciju na osnovu klase kojoj pripada najveći broj instanci iz prosleđenog skupa.

In [2021]:
def classify_data(data):

    label_column = data[:, -1]
    unique_classes, count_unique_classes = np.unique(label_column, return_counts=True)
    index = count_unique_classes.argmax()
    classification = unique_classes[index]

    return classification

Funkcija **get_potential_splits** vraća vrednosti svih potencijalnih podela, odnosno mogućih razdelnih vrednosti za svaku kolonu iz prosleđenog (**data**) skupa. Funkcija može vratiti *dictionary*, kao na primer: 

{0: [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5, 13.5, 14.5, 16.0], 1: [50.0, 56.5, 59.0, 61.5, 63.5, 66.0, 67.5...

*Dictionary* je zbog dužine prikazan samo delom. Ključ *dictionary*-ja je indeks kolone **data** skupa (prikazani su samo 0 i 1 indeksi), a vrednosti koje su pridružene svakom ključu su liste koje sadrže sve moguće razdelne vrednosti, odnosno potencijalne podele.

In [2022]:
def get_potential_splits(data):

    potential_splits = {}
    _, n_columns = data.shape

    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                potential_splits[column_index].append(potential_split)
    return potential_splits

Funkcija: **split_data** vrši podelu prosleđenog skupa instanci. Argumenti:
 - **data** - skup instanci čija se podela vrši,
 - **split_column** - indeks kolone na osnovu koje će se vršiti podela,
 - **split_value** - granična vrednost koja se koristi kao razdelnik pri podeli.
 
Funkcija vraća dve vrednosti: 
 
 -listu koja sadrži sve vrednosti koje su manje ili jednake graničniku (**split_value**),
 -listu koja sadrži sve vrednosti koje su veće od granične vrednosti koja se koristi kao razdelnik.
 

In [2023]:
def split_data(data, split_column, split_value):

    split_column_values = data[:, split_column]
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    return data_below, data_above

Funkcija **calculte_entropy** računa entropiju prosleđenog **data** skupa, u odnosu na poslednju kolonu u skupu (voditi računa da klasni atribut na osnovu koga se vrši klasifikacija bude poslednja kolona). Entropija je mera nehomogenosti skupa i kreće se u intervalu [0, 1]. Entropija ima maksimalnu vrednost (1) u slučaju da su sve instance koje čine skup ravnomerno raspoređene po svim klasama, a minimalnu (0) kada sve instance skupa pripadaju jednoj klasi. Entropija se izračunava po formuli:

$$ \sum\limits_{i = 1}^{n} p_i  * -(\log_2 p_i) $$

In [2024]:
def calculate_entropy(data):

    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))

    return entropy

Funkcija **calculate_overall_entropy** izračunava zbirnu entropiju za dva prosleđena skupa na osnovu sledeće formule:

$$ \sum\limits_{i = 1}^{2} p_i * entropija $$

In [2025]:
def calculate_overall_entropy(data_below, data_above):

    n_data_points = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below)
                       + p_data_above * calculate_entropy(data_above))

    return overall_entropy

Funkcija **determine_best_split** određuje najbolju podelu za prosleđeni skup. Argumenti:
 - **data** - Skup u kome se ispituje koja je najbolja podela. Funkcija određuje najbolju podelu na osnovu vrednosti entropije svake kolone i potencijalne grančne vrednosti individualno (bira najmanju entropiju).
 - **potential_splits** - *dictionary* koji za svaku kolonu sadrži moguće podele (podele se dobijaju tako što se prvo pozove funkcija **split_data**). Svaka potencijala podela je srednja vrednost između dve susedne vrednosti u koloni.
 
Funkcija vraća dve vrednosti: indeks kolone koju na osnovu koje je najbolje izvršiti podelu i granična vrednost koju treba koristiti pri podeli date kolone.

In [2026]:
def determine_best_split(data, potential_splits):

    overall_entropy = 999

    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value

    return best_split_column, best_split_value

Funkcija *decision_tree_algorithm* vrši generisanje stabla odluke i rekurzivnog je karaktera. Svaki poziv funkcije generiše odgovarajuće podstablo koje je oblika: 
    **{pitanje: [pozitivan_odgovor, negativan_odgovor]}**.
Samim tim stablo je realizovano u obliku *dictionary*-ja. Argumenti:
 - **df** - trening skup koji se koristi za generisanje stabla odluke,
 - **counter** - kada uzima vrednost 0, funkcija vrši raspakivanje *pandas data frame*-a u *numpy* polje, koje se koristi za potrebe poziva pomoćnih funkcija,
 - **min_samples** - Budući da se funkcija može koristiti i u svrhu generisanja jednog stabla odluke, pomoću ovog argumenta vrši se odsecanje odnosno *pruning*. Naime, ovaj argument određuje minimalnu vrednost veličine skupa ispod koje se ne vrši dalje razbijanje istog, čak i u slučaju da skup nije homogen. Ukoliko je skup instanci nehomogen, a njegova populacija manja **od min_samples** argumenta, **decision_tree_algorithm** funkcija će izvršiti klasifikaciju na osnovu prisustva najdominantnije klase u skupu. Međutim, imajući u vidu da se u slučaju *random forest algoritma* ne vrši odsecanje individualnog stabla, ovaj argument treba imati vrednost 0, jer se svako stablo koje čini ansambl generiše u potpunosti.   
 
Funkcija vraća generisano stablo odluke.

In [2027]:
def decision_tree_algorithm(df, counter=0, min_samples=0):

    if counter == 0:
        global COLUMN_HEADERS
        COLUMN_HEADERS = df.columns
        data = df.values
    else:
        data = df

    # function is recursive, therefore it should know where to stop
    if ((check_purity(data))) or (len(data) < min_samples):
        classification = classify_data(data)
        return classification
    # if data is not pure, enter recursive part
    else:
        counter += 1

        # helper functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)

        # instantiate subtree
        feature_name = COLUMN_HEADERS[split_column]
        question = "{} <= {}".format(feature_name, split_value)
        subtree = {question: []}

        # recursive part
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples)

        subtree[question].append(yes_answer)
        subtree[question].append(no_answer)

        return subtree

Funkcija *classify*, vrši klasifikaciju željene instance. Argumenti:
   - **example** - instanca čija se klasifikacija vrši,
   - **tree** - stablo koje će vršiti klasifikaciju prosleđene instance.
    
Funkcija vraća utvrđenu klasu.

In [2028]:
def classify(example, tree):

    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else:
        answer = tree[question][1]

    if not isinstance(answer, dict):
        return answer
    else:
        remaining_tree = answer
        return classify(example, remaining_tree)

Funkcija **create_bootstraped_dataset**, generiše trening skup na osnovu instanci iz skupa koji se prosleđuje kao ulazni argument **df**. Funkcija **create_list_of_bootstraped_dataset** poziva funkciju **create_bootstraped_dataset** onoliko puta koliko je trening skupova potrebno za treniranje svakog individualnih stabala odluke. Instance se prilikom kreiranja individualnog trening skupa biraju nasumično iz prosleđenog skupa, a *random forest* algoritam dozvoljava ponavljanje instanci koje će se naći u trening skupu.

In [2029]:
def create_bootstrapped_dataset(df):
    original_df_size = len(df)
    indices = np.random.randint(0, original_df_size, original_df_size)
    bootstrapped_df = df.iloc[indices]
    return bootstrapped_df

Funkcija **create_list_of_bootstraped_dataset** kreira trening skup, gde će se svaki individualni skup upotrebiti za generisanje individualnog stabla odluke. Funkcija uzima jedan ulazni argument, **set_size** , koji određuje koliko trening skupova je potrebno generisati.
Funkcija vraća listu generisanih trening skupova.

In [2030]:
def create_list_of_bootstrapped_dataset(set_size):
    bootstrapped_dfs = []
    for i in range(set_size):
        bootstrapped_dfs.append(create_bootstrapped_dataset(train_df))
    return bootstrapped_dfs

Pozivom funkcije **generate_random_forest** generišu se stabla na osnovu prosleđenog niza test skupova. Argumenti:
   - **number_of_trees** - Broj stabala koja će se generisati,
   - **bootstraped_dfs** - Niz trening skupova. Svaki trening skup iz niza koristi se za treniranje individualnog stabla odluke na odgovarajućoj poziciji.
   
Funkcija vraća niz stabala koja čine ansambl. 

In [2031]:
def generate_random_forest(number_of_trees, bootstrapped_dfs, min_s):
    decision_trees = []
    for i in range(number_of_trees):
        decision_trees.append(decision_tree_algorithm(bootstrapped_dfs[i], min_samples=min_s))
    return decision_trees

Funkcija **vote_classify** vrši klasifikaciju instance na osnovu većinskog glasanja. Argumenti:
   - **example** - Instanca čije se klasifikacija vrši,
   - **generated_trees** - Niz stabala koja vrše klasifikaciju. 

Svako stablo vrši individualnu klasifikaciju, a funkcija vraća klasu koja je dobila najveći broj glasova.

In [2032]:
def vote_classify(example, generated_trees):
    classification_votes = []

    for i in generated_trees:
        classification_votes.append(classify(example, i))

    unique_votes, number_of_votes = np.unique(classification_votes, return_counts=True)
    index = -1
    max = 0
    for i in range(len(number_of_votes)):
        if number_of_votes[i] > max:
            max = number_of_votes[i]
            index = i

    return unique_votes[index]

Funkcija **test_random_forest_accuracy**  određuje tačnost klasifikacije ansambla stabala odlučivanja. Argumenti:
 - **test_df** - Skup koji čine instance čija se klasifikacija vrši.
 - **generated_trees** - Lista stabala koja vrše klasifikaciju

Funkcija vraća tri vrednosti: ukupan broj testiranih instanci, broj korektnih klasifikacija i tačnost klasifikacije za zadati skup instanci.

In [2033]:
def test_random_forest_accuracy(test_df, generated_trees):

    number_of_correct_classifications = 0
    test_df = test_df.reset_index(drop=True)
    for i in range(len(test_df)):
        belongs_to_class = vote_classify(test_df.iloc[i], generated_trees)
        #print('belongs to class', test_df.iloc[i, -1], 'classified as: ', belongs_to_class)
        #print(i, belongs_to_class)
        if belongs_to_class == test_df.iloc[i, -1]:
            number_of_correct_classifications += 1
    #print(number_of_correct_classifications)
    #print(test_df)
    number_of_instances_tested = len(test_df)
    classification_accuracy = number_of_correct_classifications / number_of_instances_tested
    return number_of_instances_tested, number_of_correct_classifications, classification_accuracy

Poslednji deo koda odnosi se na poziv sledećih funkcija:
 - funkcije koja vrši generisanje trening skupova, 
 - funkcije koja generiše ansambl stabala na osnovu generisanih trening skupova i
 - funkcije koja vrši testiranje tačnosti klasifikacije generisanog ansambla.

In [2034]:
df = replace_missing_values(df)
best_features_sorted = determine_best_features(df)
df = keep_most_relevant_features(df, 5, best_features_sorted)
train_df, test_df = train_test_split(df, test_size=0.2)
np.random.seed(0)
bootstrapped_ds = create_list_of_bootstrapped_dataset(80)
generated_trees = generate_random_forest(80, bootstrapped_ds, 0)
#pprint(generated_trees)
#example = test_df.iloc[1]
#print(vote_classify(example, generated_trees))

number_of_instances_tested, number_of_correct_classifications, classification_accuracy = test_random_forest_accuracy(test_df, generated_trees)

print('Broj testirani instanci: ', number_of_instances_tested)
print('Broj tacnih klasifikacija: ', number_of_correct_classifications)
print('Tacnost random forest algoritma: ', classification_accuracy)


Broj testirani instanci:  154
Broj tacnih klasifikacija:  117
Tacnost random forest algoritma:  0.7597402597402597


## <center> Zaključak </center>

- Ukoliko se generiše "ansambl od samo jednog stabla", tako što će se zahtevati generisanje samo jednog trening skupa, a nakon toga generisanje samo jednog stabla, odnosno:

**bootstrapped_ds = create_list_of_bootstrapped_dataset(1)**

**generated_trees = generate_random_forest(1, bootstrapped_ds, 0)**,

a pri testiranju generisanog stabla koristimo instance koje se nalaze u skupu koji je korišćen za treniranje stabla, odnosno:

**test_random_forest_accuracy(bootstrapped_ds[0], generated_trees)**,

usled posledica prenaučenja, dobićemo da je tačnost klasifikacije 100% (odnosno 1.0).

- Međutim, ukoliko za testiranje generisanog stabla koristimo trening skup:

**test_random_forest_accuracy(test_df, generated_trees)**,

dobijamo manju tačnost klasifikacije (0.7012987012987013).

- U dajem izveštaju, date su tačnosti klasifikacije test skupa za odgovarajući broj zadržanih atributa iz inicijalno učitanog skupa podataka.
 
8 atributa

  - 1 - 0.7012987012987013
  - 10 - 0.7662337662337663
  - 20 - 0.7662337662337663
  - 30 - 0.7792207792207793
  - 40 - 0.7922077922077922
  - 50 - 0.7922077922077922
  - 60 - 0.7857142857142857
  - 70 - 0.7987012987012987
  - 80 - 0.7857142857142857

7 atributa

 - 1 - 0.6753246753246753
 - 10 - 0.7402597402597403
 - 20 - 0.7532467532467533
 - 30 - 0.7727272727272727
 - 40 - 0.7857142857142857
 - 50 - 0.7857142857142857
 - 60 - 0.7857142857142857
 - 70 - 0.7987012987012987
 - 80 - 0.7922077922077922

6 atributa

 - 1 - 0.7077922077922078
 - 10 - 0.7402597402597403
 - 20 - 0.7467532467532467
 - 30 - 0.7662337662337663
 - 40 - 0.7532467532467533
 - 50 - 0.7597402597402597
 - 60 - 0.7597402597402597
 - 70 - 0.7532467532467533
 - 80 - 0.7597402597402597

5 atributa

 - 1 - 0.7337662337662337
 - 10 - 0.7467532467532467
 - 20 - 0.7597402597402597
 - 30 - 0.7532467532467533
 - 40 - 0.7662337662337663
 - 50 - 0.7662337662337663
 - 60 - 0.7662337662337663
 - 70 - 0.7597402597402597
 - 80 - 0.7597402597402597

4 atributa

 - 1- 0.6883116883116883
 - 10 - 0.7662337662337663
 - 20 - 0.7597402597402597
 - 30 - 0.7792207792207793
 - 40 - 0.7727272727272727
 - 50 - 0.7727272727272727
 - 60 - 0.7662337662337663
 - 70 - 0.7662337662337663
 - 80 - 0.7662337662337663
 
- Kao što je u uvodu navedeno, ne postoji konkretna metoda kojom se može utvrditi optimalno rešenje, odnosno koji broj stabala daje najtačniju klasifikaciju, već se programeri odlučuju da testiraju tačnost tako što povećavaju populaciju stabala sve dok tačnost klasifikacije raste.

